In [44]:
import json
import os
import pandas as pd
import requests

In [24]:
r = requests.get('https://raw.githubusercontent.com/BioSchemas/bioschemas.github.io/master/_data/live_deployments.json')

bioschemas_deploys = json.loads(r.text)

In [27]:
resource_types = []
for eachresource in bioschemas_deploys['resources']:
    for eachprofile in eachresource['profiles']:
        resource_dict = {'name':eachresource['name'],'url':eachresource['url'],'profile':eachprofile['profileName']}
        try:
            resource_dict['sitemap']=eachresource['sitemap'],
        except:
            resource_dict['sitemap']=None
        resource_types.append(resource_dict)

resource_df = pd.DataFrame(resource_types)
frequency_df = resource_df.groupby('profile').size().reset_index(name='profile counts')
print(frequency_df)

                  profile  profile counts
0           BioChemEntity               1
1               BioSample               2
2       ChemicalSubstance               2
3       ComputationalTool               8
4   ComputationalWorkflow               1
5                  Course               3
6          CourseInstance               3
7             DataCatalog              36
8              DataRecord               1
9                 Dataset              49
10                Disease               1
11                  Event               6
12        FormalParameter               2
13                   Gene               7
14        MolecularEntity               6
15           Organization               1
16                 Person               3
17                Protein               9
18      ProteinAnnotation               2
19       ProteinStructure               2
20                 Sample               1
21       ScholarlyArticle               2
22     SequenceAnnotation         

In [51]:
gene_related = ['Gene','Protein','ProteinAnnotation','ProteinStructure','SequenceAnnotation','SequenceRange']
disease_related = ['Disease']
sample_related = ['BioSample','Sample']
dataset_related = ['DataCatalog','DataRecord','Dataset']
agent_related = ['Person','Organization']
tool_related = ['ComputationalTool','ComputationalWorkflow']
chem_related = ['ChemicalSubstance','MolecularEntity']
training_related = ['TrainingMaterial','Course','CourseInstance','Event']
study_related = ['Study']

dataset_resource = resource_df.loc[resource_df['profile'].isin(dataset_related)].copy()
dataset_resource['potential value'] = "niaid,rdp,outbreak"
dataset_resource['profile grouping'] = 'dataset'
tool_resource = resource_df.loc[resource_df['profile'].isin(tool_related)].copy()
tool_resource['potential value'] = "niaid,rdp,outbreak"
tool_resource['profile grouping'] = 'tool'

training_resource = resource_df.loc[resource_df['profile'].isin(training_related)].copy()
training_resource['potential value'] = 'rdp,outbreak'
training_resource['profile grouping'] = 'training'
study_resource = resource_df.loc[resource_df['profile'].isin(study_related)].copy()
study_resource['potential value'] = 'niaid,rdp,outbreak'
study_resource['profile grouping'] = 'study'
agent_resource = resource_df.loc[resource_df['profile'].isin(agent_related)].copy()
agent_resource['potential value'] = 'rdp'
agent_resource['profile grouping'] = 'agent'


chem_resource = resource_df.loc[resource_df['profile'].isin(chem_related)].copy()
chem_resource['potential value'] = 'mychem,biothings,translator'
chem_resource['profile grouping'] = 'chemical'
gene_resource = resource_df.loc[resource_df['profile'].isin(gene_related)].copy()
gene_resource['potential value'] = 'mygene,biothings,translator'
gene_resource['profile grouping'] = 'gene'
disease_resource = resource_df.loc[resource_df['profile'].isin(disease_related)].copy()
disease_resource['potential value'] = 'mydisease,biothings,translator'
disease_resource['profile grouping'] = 'disease'
sample_resource = resource_df.loc[resource_df['profile'].isin(sample_related)].copy()
sample_resource['potential value'] = 'biothings,translator'
sample_resource['profile grouping'] = 'sample'

In [53]:
export_df = pd.concat((dataset_resource,tool_resource,training_resource,agent_resource,
                       study_resource,chem_resource,gene_resource,disease_resource,
                       sample_resource),ignore_index=True)

export_df.to_csv('bioschemas_deploys.tsv',sep='\t',header=True)